In [79]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras.models import load_model
from uproot_io import Events, View

In [80]:
x_train = np.load("/Users/mohammed/code/neutrinos-dune-files/dataSmall2/x_train.npy")
y_train = np.load("/Users/mohammed/code/neutrinos-dune-files/dataSmall2/y_train.npy")

x_val = np.load("/Users/mohammed/code/neutrinos-dune-files/dataSmall2/x_val.npy")
y_val = np.load("/Users/mohammed/code/neutrinos-dune-files/dataSmall2/y_val.npy")

x_test = np.load("/Users/mohammed/code/neutrinos-dune-files/dataSmall2/x_test.npy")
y_test = np.load("/Users/mohammed/code/neutrinos-dune-files/dataSmall2/y_test.npy")

In [72]:
# model 1
num_classes = 3
input_layer = keras.layers.Input(x_train[0].shape)
x = keras.layers.Conv2D(32, (3,3), activation='relu')(input_layer) 
x = keras.layers.MaxPooling2D((2,2))(x) 
x = keras.layers.Dropout(0.5)(x) 
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(num_classes, activation='softmax')(x) 
cnn_model1 = keras.Model(input_layer, x)
cnn_model1.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 128, 128, 1)]     0         
                                                                 
 conv2d_26 (Conv2D)          (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 63, 63, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_20 (Dropout)        (None, 63, 63, 32)        0         
                                                                 
 flatten_11 (Flatten)        (None, 127008)            0         
                                                                 
 dense_12 (Dense)            (None, 3)                 381027    
                                                           

In [76]:
batch_size = 256
epochs = 7
learning_rate = 0.001

In [77]:
loss_function = keras.losses.categorical_crossentropy
optimiser = keras.optimizers.legacy.Adam(learning_rate=learning_rate)
cnn_model1.compile(loss=loss_function, optimizer=optimiser, metrics=['accuracy'])

In [78]:
cnn_model1.fit(x = x_train, y = y_train, batch_size = batch_size, epochs = epochs,
              validation_data = (x_val, y_val), verbose = 1)

Epoch 1/7
88/88 [==============================] - 23s 257ms/step - loss: 0.3201 - accuracy: 0.8759 - val_loss: 0.8931 - val_accuracy: 0.6979
Epoch 2/7
88/88 [==============================] - 22s 249ms/step - loss: 0.2880 - accuracy: 0.8907 - val_loss: 0.9277 - val_accuracy: 0.6928
Epoch 3/7
88/88 [==============================] - 22s 249ms/step - loss: 0.2716 - accuracy: 0.9000 - val_loss: 0.9541 - val_accuracy: 0.6949
Epoch 4/7
88/88 [==============================] - 24s 268ms/step - loss: 0.2575 - accuracy: 0.9026 - val_loss: 0.9966 - val_accuracy: 0.6858
Epoch 5/7
88/88 [==============================] - 26s 293ms/step - loss: 0.2464 - accuracy: 0.9053 - val_loss: 1.0292 - val_accuracy: 0.6901
Epoch 6/7
88/88 [==============================] - 24s 278ms/step - loss: 0.2362 - accuracy: 0.9100 - val_loss: 1.1009 - val_accuracy: 0.6874
Epoch 7/7
88/88 [==============================] - 22s 256ms/step - loss: 0.2309 - accuracy: 0.9121 - val_loss: 1.1219 - val_accuracy: 0.6855


In [34]:
# Make a list of incorrect classifications
incorrect_indices = []
# Let's look at the whole test dataset, but you can reduce this to 1000 or so
# if you want run more quickly
n_images_to_check = x_test.shape[0]
# Use the CNN to predict the classification of the images. It returns an array
# containing the 10 class scores for each image. It is best to write this code
# using the array notation x[:i] that means use all values of x up until
# the index i, such that if you changed the number of images above then it all
# still works efficiently
raw_predictions = cnn_model1.predict(x = x_test[:n_images_to_check], batch_size = batch_size)
for i in range(0,n_images_to_check):
  # Remember the raw output from the CNN gives us an array of scores. We want
  # to select the highest one as our prediction. We need to do the same thing
  # for the truth too since we converted our numbers to a categorical
  # representation earlier. We use the np.argmax() function for this
  prediction = np.argmax(raw_predictions[i])
  truth = np.argmax(y_test[i])
  if prediction != truth:
    incorrect_indices.append([i,prediction,truth])
print('Number of images that were incorrectly classified =',len(incorrect_indices))

88/88 [==============================] - 3s 37ms/step
Number of images that were incorrectly classified = 3300


In [35]:
a = (len(x_test)-3300)/len(x_test)
a

0.7038499506416585

In [36]:
loss, accuracy = cnn_model1.evaluate(x_test, y_test)

349/349 [==============================] - 6s 17ms/step - loss: 0.7611 - accuracy: 0.7038


In [81]:
from tensorflow.keras import layers, models, regularizers
cnn_model2 = models.Sequential([
        layers.Input(shape=x_train[0].shape),
        layers.Rescaling(1./255),
        
        # First Conv Block with BatchNorm and MaxPooling
        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        
        # Second Conv Block
        layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.4),
        
        # Third Conv Block
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.5),
        
        # Classifier
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
cnn_model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 128, 128, 1)       0         
                                                                 
 conv2d_27 (Conv2D)          (None, 128, 128, 64)      640       
                                                                 
 batch_normalization_21 (Ba  (None, 128, 128, 64)      256       
 tchNormalization)                                               
                                                                 
 conv2d_28 (Conv2D)          (None, 128, 128, 64)      36928     
                                                                 
 batch_normalization_22 (Ba  (None, 128, 128, 64)      256       
 tchNormalization)                                               
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 64, 64, 64)       

In [57]:
subset_size = 1000  # For example, 1000 samples

# Ensure reproducibility
np.random.seed(42)

# Randomly select indices for the subset
subset_indices = np.random.choice(x_train.shape[0], subset_size, replace=False)

# Extract the subset of data and labels
x_train_subset = x_train[subset_indices]
y_train_subset = y_train[subset_indices]

In [86]:
batch_size = 64
epochs = 2
learning_rate = 0.001

In [87]:
loss_function = keras.losses.categorical_crossentropy
optimiser = keras.optimizers.legacy.Adam(learning_rate=learning_rate)
cnn_model2.compile(loss=loss_function, optimizer=optimiser, metrics=['accuracy'])

In [ ]:
cnn_model2.fit(x = x_train, y = y_train, batch_size = batch_size, epochs = epochs,
              validation_data = (x_val, y_val), verbose = 1)

Epoch 1/2
349/349 [==============================] - 344s 984ms/step - loss: 0.8302 - accuracy: 0.6857 - val_loss: 1.7674 - val_accuracy: 0.4058
Epoch 2/2
285/349 [=======================>......] - ETA: 7:07 - loss: 0.6765 - accuracy: 0.7267

In [61]:
loss, accuracy = cnn_model2.evaluate(x_test, y_test)

349/349 [==============================] - 40s 114ms/step - loss: 7.5386 - accuracy: 0.4141


In [62]:
model = load_model('/Users/mohammed/code/neutrinos-dune-files/modelv2')

In [63]:
x_train = np.load("/Users/mohammed/code/neutrinos-dune-files/dataBig/x_train.npy")
y_train = np.load("/Users/mohammed/code/neutrinos-dune-files/dataBig/y_train.npy")

x_val = np.load("/Users/mohammed/code/neutrinos-dune-files/dataBig/x_val.npy")
y_val = np.load("/Users/mohammed/code/neutrinos-dune-files/dataBig/y_val.npy")

x_test = np.load("/Users/mohammed/code/neutrinos-dune-files/dataBig/x_test.npy")
y_test = np.load("/Users/mohammed/code/neutrinos-dune-files/dataBig/y_test.npy")

In [65]:
loss, accuracy = model.evaluate(x_test, y_test)

349/349 [==============================] - 22s 62ms/step - loss: 18.8156 - accuracy: 0.5949
